### About
This notebook contains simple, toy examples to help you get started with FairMLHealth tool usage. This same content is mirrored in the repository's main [README](../README.md).

### Example Setup

In [27]:
from fairmlhealth import model_comparison as fhmc, reports

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier


np.random.seed(547)
X = pd.DataFrame({'col1': np.random.randint(1, 50, 16), 
                  'col2': np.random.randint(1, 50, 16),
                  'col3': np.random.randint(1, 50, 16),
                  'gender': [0, 1]*8, 
                  'ethnicity': [1, 1, 0, 0]*4,
                  'other': [1, 0, 0, 1]*4
                 })
y = pd.Series([1, 1, 1, 0, 1, 1, 0, 0]*2, name='y')
X_train, y_train= X.iloc[0:5], y.iloc[0:5]
X_test, y_test = X.iloc[5:16], y.iloc[5:16]

#Train models
model_1 = BernoulliNB().fit(X_train, y_train)
model_2 = DecisionTreeClassifier().fit(X_train, y_train)

# Deterimine your set of protected attributes
prtc_attr = X_test['gender']

# Specify either a dict or a list of trained models to compare
model_dict = {'model_1': model_1, 'model_2': model_2}


In [28]:
display(X)

,col1,col2,col3,gender,ethnicity,other
0,48,36,20,0,1,1
1,24,48,44,1,1,0
2,19,18,44,0,0,0
3,38,29,49,1,0,1
4,45,33,23,0,1,1
5,35,43,5,1,1,0
6,24,33,12,0,0,0
7,39,39,46,1,0,1
8,6,11,31,0,1,1
9,36,21,16,1,1,0


### Model Measurement
The primary feature of this library is the model comparison tool. The current version supports assessment of binary prediction models through use of the measure_models and compare_models functions.

Measure_model is designed to generate a report of multiple fairness metrics for a single model. Here it is shown wrapped in a "flag" function to emphasize values that are outside of the "fair" range. 

In [29]:
# Generate a pandas dataframe of measures
fhmc.measure_model(X_test, y_test, prtc_attr, model_1, flag_oor=True)


### Evaluating

FairMLHealth now also includes stratified reporting features to aid in identifying the source of unfairness or other bias: data reports, performance reports, and bias reports. Note that these stratified reports can evaluate multiple features at once, and that there are two options for identifying which features to assess.

Note that the flag tool has not yet been updated to work with stratified reports.

#### Stratified Data Reports

The data reporter is shown below with each of the two data argument options. It evaluates basic statistics specific to each feature-value, in addition to relative statistics for the target value. Since the reporter can evaluate many features at once, it can be a useful option for identifying patterns of bias either alone or in concert with other (e.g., visual methods).

In [30]:
# Arguments Option 1: pass full set of data, subsetting with *features* argument
reports.data_report(X_test, y_test, features=['gender'])

,Feature Name,Feature Value,Obs.,Entropy,Missing Values,Value Prevalence,y Mean,y Median,y Std. Dev.
0,ALL FEATURES,ALL VALUES,11,NaN,0,1.000000,0.545455,1.0,0.522233
1,gender,0,5,0.99403,0,0.454545,0.600000,1.0,0.547723
2,gender,1,6,0.99403,0,0.545455,0.500000,0.5,0.547723


In [31]:
# Multiple targets can also be passed to the data report 
reports.data_report(X_test, X_test, features=['gender', 'col1'], targets=['col2', 'col3'])

Feature Name Feature Value  Obs.  Entropy  Missing Values  \
0  ALL FEATURES    ALL VALUES    11      NaN               0   
1        gender             0     5  0.99403               0   
2        gender             1     6  0.99403               0   

   Value Prevalence    y Mean  y Median  y Std. Dev.  
0          1.000000  0.545455       1.0     0.522233  
1          0.454545  0.600000       1.0     0.547723  
2          0.545455  0.500000       0.5     0.547723

In [32]:
# The "ALL FEATURES" overview column can be turned off via the add_overview argument
reports.data_report(X_test[['gender']], y_test, add_overview=False)

,Feature Name,Feature Value,Obs.,Entropy,Missing Values,Value Prevalence,y Mean,y Median,y Std. Dev.
0,gender,0,5,0.99403,0,0.454545,0.6,1.0,0.547723
1,gender,1,6,0.99403,0,0.545455,0.5,0.5,0.547723


In [33]:
# Multiple targets can also be passed to the data report 
reports.data_report(X_test, X_test, features=['gender', 'col1'], targets=['col2', 'col3'])

,Feature Name,Feature Value,Obs.,Entropy,Missing Values,Value Prevalence,col2 Mean,col2 Median,col2 Std. Dev.,col3 Mean,col3 Median,col3 Std. Dev.
0,ALL FEATURES,ALL VALUES,11,NaN,0,1.000000,29.818182,33.0,13.075306,26.363636,22.0,14.780208
1,gender,0,5,0.994030,0,0.454545,27.400000,33.0,15.852445,27.800000,31.0,12.755391
2,gender,1,6,0.994030,0,0.545455,31.833333,34.5,11.409061,25.166667,19.0,17.405938
3,col1,2,1,3.277613,0,0.090909,30.000000,30.0,NaN,47.000000,47.0,NaN
4,col1,6,1,3.277613,0,0.090909,11.000000,11.0,NaN,31.000000,31.0,NaN
5,col1,20,1,3.277613,0,0.090909,16.000000,16.0,NaN,15.000000,15.0,NaN
6,col1,24,1,3.277613,0,0.090909,33.000000,33.0,NaN,12.000000,12.0,NaN
7,col1,35,1,3.277613,0,0.090909,43.000000,43.0,NaN,5.000000,5.0,NaN
8,col1,36,1,3.277613,0,0.090909,21.000000,21.0,NaN,16.000000,16.0,NaN
9,col1,37,2,3.277613,0,0.181818,41.500000,41.5,2.121320,32.000000,32.0,18.384776


#### Stratified Performance Reports

The stratified perofrmance reporter evaluates model performance specific to each feature-value subset. If prediction probabilities are available to the model, additional ROC AUC and PR AUC values will be included.

In [34]:
reports.performance_report(X_test[['gender']], y_test, model_1.predict(X_test))

,Feature Name,Feature Value,Obs.,Target Mean,Pred. Mean,Accuracy,FPR,Precision,TPR
0,ALL FEATURES,ALL VALUES,11.0,0.545455,1.0,0.545455,1.0,0.545455,1.0
1,gender,0,5.0,0.600000,1.0,0.600000,1.0,0.600000,1.0
2,gender,1,6.0,0.500000,1.0,0.500000,1.0,0.500000,1.0


In [35]:
reports.performance_report(X_test[['gender']], y_test, 
                           model_1.predict(X_test), y_prob=model_1.predict_proba(X_test)[:,1])

,Feature Name,Feature Value,Obs.,Target Mean,Pred. Mean,Accuracy,FPR,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,11.0,0.545455,1.0,0.545455,1.0,NaN,0.545455,0.766667,1.0
1,gender,0,5.0,0.600000,1.0,0.600000,1.0,0.333333,0.600000,0.833333,1.0
2,gender,1,6.0,0.500000,1.0,0.500000,1.0,0.000000,0.500000,1.000000,1.0


#### Stratified Bias Fairness Reports

The stratified bias reporter presents model bias specific to each feature-value subset. Inspired by common measures of fairness, the reporter assumes each feature-value as the "privileged" group relative to all other possible values for the feature. For example, row 3 in the table below displaying measures of "col1" value of "2" where 2 is considered to be the privileged group and all other values (1, 2, 45, and 50) are considered unprivileged.

To simplify the report, fairness measures have been simplified to their component parts. For example, measures of Equalized Odds can be determined by combining the True Positive Rate (TPR) Ratios & Differences with False Positive Rate (FPR) Ratios & Differences.

See also: [Fairness Quick References](../docs/Fairness_Quick_References.pdf) and the [Tutorial for Evaluating Fairness in Binary Classification](./Tutorial-EvaluatingFairnessInBinaryClassification.ipynb)

In [36]:
reports.bias_report(X_test[['gender', 'col3']], y_test, model_1.predict(X_test), flag_oor=True)

~/repos/fairMLHealth/fairmlhealth/utils.py:67: UserWarning: Possible error in column(s) ['col3']. The ratio is ill-defined and being set to 0.0 because 'false_positive_rate' for privileged samples is 0.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,Feature Name,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,TPR Diff,TPR Ratio
0,gender,0.000000,1.000000,-0.100000,0.833333,0.000000,1.000000
1,gender,0.000000,1.000000,0.100000,1.200000,0.000000,1.000000
2,col3,0.000000,1.000000,0.600000,0.000000,1.000000,0.000000
3,col3,1.000000,0.000000,-0.500000,0.500000,0.000000,1.000000
4,col3,1.000000,0.000000,-0.500000,0.500000,0.000000,1.000000
5,col3,1.000000,0.000000,-0.500000,0.500000,0.000000,1.000000
6,col3,0.000000,1.000000,0.600000,0.000000,1.000000,0.000000
7,col3,1.000000,0.000000,-0.500000,0.500000,0.000000,1.000000
8,col3,1.000000,0.000000,-0.500000,0.500000,0.000000,1.000000
9,col3,0.000000,1.000000,0.600000,0.000000,1.000000,0.000000


In [37]:
reports.bias_report(X_test[['gender', 'col3']], y_test, model_1.predict(X_test), 
                    flag_oor=True, cohorts=X_test['other'])

~/repos/fairMLHealth/fairmlhealth/utils.py:67: UserWarning: Possible error in column(s) ['gender']. The ratio is ill-defined and being set to 0.0 because 'false_positive_rate' for privileged samples is 0.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
~/repos/fairMLHealth/fairmlhealth/utils.py:67: UserWarning: Possible error in column(s) ['col3']. The ratio is ill-defined and being set to 0.0 because 'false_positive_rate' for privileged samples is 0.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
~/repos/fairMLHealth/fairmlhealth/utils.py:67: UserWarning: Possible error in column(s) ['gender']. The ratio is ill-defined and being set to 0.0 because 'precision_score' for privileged samples is 0.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
~/repos/fairMLHealth/fairmlhealth/utils.py:67: UserWarning: Possible error in column(s) ['gender']. The ratio is ill-defined and being set to 0.0 because 'true_positive_rate' for privileged samples is 0.

  warn(f"Possible err

### Comparing Results for Multiple Models

The compare_models feature can be used to generate side-by-side fairness comparisons of multiple models. Model performance metrics such as accuracy and precision are also provided to facilitate comparison.   

Below is an example output comparing the two example models defined above. Missing values have been added for metrics requiring prediction probabilities, which the second model does not have (note the warning below).

In [38]:
# Pass the data and models to the compare models function, as above
fhmc.compare_models(X_test, y_test, prtc_attr, model_dict, flag_oor=True)

The compare_models function can also be used to measure two different protected attributes. Protected attributes are measured separately and cannot yet be combined together with this tool.

In [39]:
fhmc.compare_models(X_test, y_test, 
                     [X_test['gender'], X_test['ethnicity']], 
                      {'gender':model_1, 'ethnicity':model_1},
                   flag_oor=True)
